# 自动求导机制

在tensorflow中自动求导机制非常简单，核心在于`tf.GradientTap`。我们首先会通过一个例子来给展示如何快速计算变量的梯度。

## 1. 例子：如何计算变量的梯度

In [20]:
import tensorflow as tf

x = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
    # y = 3^2
    y = tf.square(x)
y_grad = tape.gradient(y, x)

print(f"y值：{y}")
print(f'y对x计算的梯度为：{y_grad}')

y值：9.0
y对x计算的梯度为：6.0


对于上述代码有几处知识点需要注意：
1. 在定义Variable的时候，`initial_value`的值必须是浮点值
1. shdi f

## 2. 例子：一元二次方程参数求解

问题定义：y = x * w + b

给定x和y，随机初始化参数w和b，然后通过梯度的更新来求解最优化w和b的权重

In [8]:
import tensorflow as tf

# 定义x和y训练数据
x = tf.random.uniform(shape=())
y = 2 * x + 3

# 定义需要计算的相关参数

w = tf.Variable(tf.random.uniform(shape=()))
b = tf.Variable(tf.random.uniform(shape=()))

In [11]:
parameters = [w, b]
with tf.GradientTape() as tap:
    y_predict = x * w + b

## 3. 矩阵参数求解

In [38]:
# 定义输出
x = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])

# 定义参数
w = tf.Variable(initial_value=[[1.], [2.]], name="w")
b = tf.Variable(initial_value=1., name="b")

# 需要学习的参数列表
parameters = [w, b]

# 执行计算过程，同时保留梯度信息
with tf.GradientTape() as tape:
    loss = tf.reduce_sum(tf.square(tf.matmul(x, w) + b -y))

# 计算之后的梯度信息
parameter_gradients = tape.gradient(loss, parameters)

# 最终得到的梯度信息是和参数列表一一对应的
assert len(parameter_gradients) == len(parameters)

# 循环打印变量和梯度数据
for i in range(len(parameters)):
    print(f"source variable: {parameters[i].name} -> {parameters[i].numpy()} graident: {parameter_gradients[i]} ")

source variable: w:0 -> [[1.]
 [2.]] graident: [[ 70.]
 [100.]] 
source variable: b:0 -> 1.0 graident: 30.0 


## 线性回归计算

In [78]:
from tqdm.notebook import tqdm, tqdm_notebook

# 定义基础输入
x = tf.random.uniform(shape=(2,3))
y = 3 * x + 2

# 定义3和2对应的参数w, b
w = tf.Variable(initial_value=tf.random.uniform(shape=()))
b = tf.Variable(initial_value=tf.random.uniform(shape=()))

# 需要学习的参数
parameters = [w, b]
epochs = 1000
# 定义优化器
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

with tqdm(total=epochs) as bar:
    for i in range(epochs):
        # 在tape变量作用域内是可以将所有梯度信息保留着
        with tf.GradientTape() as tape:
            y_hat = x * w + b
            loss = tf.reduce_sum(tf.square(y_hat, y))
        # 从磁带（tape）中梯度出参数对应的梯度
        gradients = tape.gradient(loss, parameters)
        if i % 10 == 0:
            bar.set_description(f"loss: {loss}")
        bar.update(1)
        # 优化器根据参数对应的梯度信息，更新参数值
        optimizer.apply_gradients(grads_and_vars=zip(gradients, parameters))